# Monitor jobs

In [1]:
import boto3
batch = boto3.client('batch')
jobid = '71e29f77-5b8b-4242-a78b-154dc06d05b9'

In [2]:
import pandas as pd
df = pd.read_csv('s3://dinosar/processing/uturuncu/A156-EEtest/pairs.txt', names=['s3path'])

In [3]:
df['pair'] = df.s3path.str[-21:]

In [4]:
df.head()

,s3path,pair
0,s3://dinosar/processing/uturuncu/A156-EEtest/i...,int-20191002-20190902
1,s3://dinosar/processing/uturuncu/A156-EEtest/i...,int-20190902-20190803


In [5]:
def get_logid(i, jobid):
    metadata = batch.describe_jobs(jobs=[f'{jobid}:{i}'])
    logid = metadata['jobs'][0]['container']['logStreamName']
    return logid

In [6]:
df['logid'] = [get_logid(i,jobid) for i in df.index.values]

In [7]:
df

,s3path,pair,logid
0,s3://dinosar/processing/uturuncu/A156-EEtest/i...,int-20191002-20190902,uturuncu-array/default/222db44c-3a23-4e78-b558...
1,s3://dinosar/processing/uturuncu/A156-EEtest/i...,int-20190902-20190803,uturuncu-array/default/226c3b17-04d1-45cb-a849...


In [8]:
def get_logurl(logid, jobid):
    logurl = logid.replace('/','$252F')
    url = f'https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logsV2:log-groups/log-group/$252Faws$252Fbatch$252Fjob/log-events/{logurl}'
    return url

In [9]:
get_logurl(df.logid[1], jobid)

'https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logsV2:log-groups/log-group/$252Faws$252Fbatch$252Fjob/log-events/uturuncu-array$252Fdefault$252F226c3b17-04d1-45cb-a849-a16871f5f801'

In [10]:
def get_loglink(df, pair):
    ''' pass dataframe and pair '''
    logid = df[df.pair==pair].logid[0]
    logurl = logid.replace('/','$252F')
    url = f'https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logsV2:log-groups/log-group/$252Faws$252Fbatch$252Fjob/log-events/{logurl}'
    return url

In [11]:
get_loglink(df, 'int-20191002-20190902')

'https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logsV2:log-groups/log-group/$252Faws$252Fbatch$252Fjob/log-events/uturuncu-array$252Fdefault$252F222db44c-3a23-4e78-b558-b5b811ddc892'

In [12]:
def get_log(logid, n=20):
    ''' last n messages from log '''
    cloudlogs = boto3.client('logs')
    response = cloudlogs.get_log_events(logGroupName='/aws/batch/job', logStreamName=logid)
    with open('log.txt', 'w') as f:
        lastLines = [e['message'] for e in response['events'][-n:]]
        print('\n'.join(lastLines))

In [13]:
get_log(df.logid[1])

Child returned 0
Child returned 0
Child returned 0
CWD:  /tmp/tmp.BKcIPYkUS
AWS_BATCH_JOB_ID 71e29f77-5b8b-4242-a78b-154dc06d05b9:1
AWS_BATCH_JQ_NAME uturuncu-queue
AWS_BATCH_JOB_ARRAY_INDEX 1
AWS_BATCH_JOB_ATTEMPT 1
AWS_BATCH_CE_NAME uturuncu
aws s3 sync s3://dinosar/processing/uturuncu/A156-EEtest/int-20190902-20190803 .
aws s3 sync s3://dinosar/processing/uturuncu/dem .
aria2c -c -x 8 -s 8 -i download-links.txt
topsApp.py --steps 2>&1 | tee topsApp.log
cp topsApp.xml topsApp.log topsProc.xml download-links.txt merged/
aws s3 sync merged s3://dinosar/results/uturuncu/A156-EEtest/int-20190902-20190803
Child returned 0
Child returned 0
aws s3 cp s3://dinosar/processing/uturuncu/A156-EEtest/pairs.txt .
Batch index: 1, Processing pair: s3://dinosar/processing/uturuncu/A156-EEtest/int-20190902-20190803
/tmp/tmp.BKcIPYkUS/sentinel1-uturuncu-master/fetch_and_run/run_interferogram_aws.py -i s3://dinosar/processing/uturuncu/A156-EEtest/int-20190902-20190803 -d s3://dinosar/processing/uturuncu